In [94]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [95]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [96]:
df=pd.read_csv('/content/drive/MyDrive/CP-3/ODI_Match_Data.csv')

In [97]:
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id
0,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.1,India,Australia,RD Gaikwad,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1389389
1,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.2,India,Australia,RD Gaikwad,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1389389
2,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.3,India,Australia,RD Gaikwad,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1389389
3,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.4,India,Australia,RD Gaikwad,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1389389
4,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.5,India,Australia,RD Gaikwad,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1389389


In [98]:
df.shape

(1265103, 23)

In [99]:
#creating total runs column to calculate runs on each ball incusive of extras
df["total_runs"] = df["runs_off_bat"] + df["extras"]

In [100]:
#calculating cumulative runs
df["cumulative_runs"] = df.groupby(["match_id", "innings"])["total_runs"].cumsum()

In [101]:
#creating over and balls from ball column which is in 0.1, 0.2 format for first over
df["ball"] = df["ball"].astype(str)
df[["over", "ball_num"]] = df["ball"].str.split(".", expand=True).astype(int)
df["ball"] = df["ball"].astype(float)
df["over"] = df["over"] + 1
df["ball_left"] = 306 - (df["over"]*6 + df["ball_num"])

In [102]:
df['wicket'] = df['wicket_type'].notnull().astype(int).groupby([df['match_id'], df['batting_team']]).cumsum()

In [103]:
df["wickets_left"] = df.groupby("match_id")["wicket"].apply(lambda x: 10 - x.cumsum()).reset_index(level=0, drop=True)

In [104]:
#creating run rate column
df["current_run_rate"] = df.groupby(["match_id","innings"]).apply(lambda x: (x["cumulative_runs"]*6)/(300-x["ball_left"])).reset_index(level=[0,1], drop=True)

In [105]:
groups=df.groupby('match_id')

match_ids=df['match_id'].unique()
last_five=[]
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs_off_bat'].values.tolist())
df['last_five']=last_five
df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,cricsheet_id,total_runs,cumulative_runs,over,ball_num,ball_left,wicket,wickets_left,current_run_rate,last_five
0,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.1,India,Australia,RD Gaikwad,Shubman Gill,...,1389389,4,4,1,1,299,0,10,24.000000,NaN
1,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.2,India,Australia,RD Gaikwad,Shubman Gill,...,1389389,0,4,1,2,298,0,10,12.000000,NaN
2,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.3,India,Australia,RD Gaikwad,Shubman Gill,...,1389389,4,8,1,3,297,0,10,16.000000,NaN
3,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.4,India,Australia,RD Gaikwad,Shubman Gill,...,1389389,0,8,1,4,296,0,10,12.000000,NaN
4,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.5,India,Australia,RD Gaikwad,Shubman Gill,...,1389389,0,8,1,5,295,0,10,9.600000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265098,64814,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,...,64814,0,216,43,6,42,8,-1427,5.023256,26.0
1265099,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,...,64814,1,217,44,1,41,8,-1435,5.027027,27.0
1265100,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,...,64814,0,217,44,2,40,9,-1444,5.007692,23.0
1265101,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,...,64814,2,219,44,3,39,9,-1453,5.034483,25.0


### So this is that dataset we have. We need to create some columns and extract few to get the desired data. Eventually we want our data to have columns:

- batting team
- bowling team
- city
- current_score
- balls left
- wickets_left
- current_run_rate
- last five

### Now we already have few columns as we want it in our dataset. Batting team and bowling team data we already have. we also have city column but it has some null values which we need to figure out how to handle that. For rest all we need do some manipulation.

### Now we will start our feature extraction from the city column. To fill the empty values we will use venue column.



In [106]:
df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,cricsheet_id,total_runs,cumulative_runs,over,ball_num,ball_left,wicket,wickets_left,current_run_rate,last_five
0,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.1,India,Australia,RD Gaikwad,Shubman Gill,...,1389389,4,4,1,1,299,0,10,24.000000,NaN
1,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.2,India,Australia,RD Gaikwad,Shubman Gill,...,1389389,0,4,1,2,298,0,10,12.000000,NaN
2,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.3,India,Australia,RD Gaikwad,Shubman Gill,...,1389389,4,8,1,3,297,0,10,16.000000,NaN
3,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.4,India,Australia,RD Gaikwad,Shubman Gill,...,1389389,0,8,1,4,296,0,10,12.000000,NaN
4,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.5,India,Australia,RD Gaikwad,Shubman Gill,...,1389389,0,8,1,5,295,0,10,9.600000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265098,64814,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,...,64814,0,216,43,6,42,8,-1427,5.023256,26.0
1265099,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,...,64814,1,217,44,1,41,8,-1435,5.027027,27.0
1265100,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,...,64814,0,217,44,2,40,9,-1444,5.007692,23.0
1265101,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,...,64814,2,219,44,3,39,9,-1453,5.034483,25.0


### Coming to current_runs column which is very easy to extract from the runs column. A simple cumsum() function (used to find the cumulative sum of a column) will do the work for us.



In [107]:
df['current_score'] = df.groupby('match_id')['runs_off_bat', 'extras'].cumsum().sum(axis=1)

In [108]:
df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,total_runs,cumulative_runs,over,ball_num,ball_left,wicket,wickets_left,current_run_rate,last_five,current_score
0,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.1,India,Australia,RD Gaikwad,Shubman Gill,...,4,4,1,1,299,0,10,24.000000,NaN,4
1,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.2,India,Australia,RD Gaikwad,Shubman Gill,...,0,4,1,2,298,0,10,12.000000,NaN,4
2,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.3,India,Australia,RD Gaikwad,Shubman Gill,...,4,8,1,3,297,0,10,16.000000,NaN,8
3,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.4,India,Australia,RD Gaikwad,Shubman Gill,...,0,8,1,4,296,0,10,12.000000,NaN,8
4,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.5,India,Australia,RD Gaikwad,Shubman Gill,...,0,8,1,5,295,0,10,9.600000,NaN,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265098,64814,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,...,0,216,43,6,42,8,-1427,5.023256,26.0,470
1265099,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,...,1,217,44,1,41,8,-1435,5.027027,27.0,471
1265100,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,...,0,217,44,2,40,9,-1444,5.007692,23.0,471
1265101,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,...,2,219,44,3,39,9,-1453,5.034483,25.0,473


### Now our next target is to create a ‘balls_left’ column for which firstly we would be creating to new columns: ‘overs’ and ‘balls’ which tells us how many overs have been completed and how many balls of the current over has been bowled respectively. The code is very simple for that.

In [109]:
df['over']=df['ball'].apply(lambda x:str(x).split(".")[0])
df['ball_no']=df['ball'].apply(lambda x:str(x).split(".")[1])

In [110]:
df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,cumulative_runs,over,ball_num,ball_left,wicket,wickets_left,current_run_rate,last_five,current_score,ball_no
0,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.1,India,Australia,RD Gaikwad,Shubman Gill,...,4,0,1,299,0,10,24.000000,NaN,4,1
1,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.2,India,Australia,RD Gaikwad,Shubman Gill,...,4,0,2,298,0,10,12.000000,NaN,4,2
2,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.3,India,Australia,RD Gaikwad,Shubman Gill,...,8,0,3,297,0,10,16.000000,NaN,8,3
3,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.4,India,Australia,RD Gaikwad,Shubman Gill,...,8,0,4,296,0,10,12.000000,NaN,8,4
4,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.5,India,Australia,RD Gaikwad,Shubman Gill,...,8,0,5,295,0,10,9.600000,NaN,8,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265098,64814,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,...,216,42,6,42,8,-1427,5.023256,26.0,470,6
1265099,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,...,217,43,1,41,8,-1435,5.027027,27.0,471,1
1265100,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,...,217,43,2,40,9,-1444,5.007692,23.0,471,2
1265101,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,...,219,43,3,39,9,-1453,5.034483,25.0,473,3


### Now by using a simple formula we can create a ‘balls_bowled’ column that is how many balls have been bowled. Formula would be

``` balls_bowled = (overs * 6) + balls```

In [111]:
df['balls_bowled']=(df['over'].astype('int')*6) + df['ball_no'].astype('int')

In [112]:
df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,over,ball_num,ball_left,wicket,wickets_left,current_run_rate,last_five,current_score,ball_no,balls_bowled
0,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.1,India,Australia,RD Gaikwad,Shubman Gill,...,0,1,299,0,10,24.000000,NaN,4,1,1
1,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.2,India,Australia,RD Gaikwad,Shubman Gill,...,0,2,298,0,10,12.000000,NaN,4,2,2
2,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.3,India,Australia,RD Gaikwad,Shubman Gill,...,0,3,297,0,10,16.000000,NaN,8,3,3
3,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.4,India,Australia,RD Gaikwad,Shubman Gill,...,0,4,296,0,10,12.000000,NaN,8,4,4
4,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.5,India,Australia,RD Gaikwad,Shubman Gill,...,0,5,295,0,10,9.600000,NaN,8,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265098,64814,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,...,42,6,42,8,-1427,5.023256,26.0,470,6,258
1265099,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,...,43,1,41,8,-1435,5.027027,27.0,471,1,259
1265100,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,...,43,2,40,9,-1444,5.007692,23.0,471,2,260
1265101,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,...,43,3,39,9,-1453,5.034483,25.0,473,3,261


### And now finally we can create our desired column ‘balls_left’ by subtracting balls_bowled from 120 because there are total 120 balls in an innings. sometimes because of extras (wide, no ball …) the ball count exceeds 120 so in such case we can simply give the value of 0.

In [113]:
df['balls_left']=300-df['balls_bowled']
df['balls_left']=df['balls_left'].apply(lambda x:0 if x<0 else x)

In [114]:
df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,ball_num,ball_left,wicket,wickets_left,current_run_rate,last_five,current_score,ball_no,balls_bowled,balls_left
0,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.1,India,Australia,RD Gaikwad,Shubman Gill,...,1,299,0,10,24.000000,NaN,4,1,1,299
1,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.2,India,Australia,RD Gaikwad,Shubman Gill,...,2,298,0,10,12.000000,NaN,4,2,2,298
2,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.3,India,Australia,RD Gaikwad,Shubman Gill,...,3,297,0,10,16.000000,NaN,8,3,3,297
3,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.4,India,Australia,RD Gaikwad,Shubman Gill,...,4,296,0,10,12.000000,NaN,8,4,4,296
4,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.5,India,Australia,RD Gaikwad,Shubman Gill,...,5,295,0,10,9.600000,NaN,8,5,5,295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265098,64814,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,...,6,42,8,-1427,5.023256,26.0,470,6,258,42
1265099,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,...,1,41,8,-1435,5.027027,27.0,471,1,259,41
1265100,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,...,2,40,9,-1444,5.007692,23.0,471,2,260,40
1265101,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,...,3,39,9,-1453,5.034483,25.0,473,3,261,39


### Now if we look at the ‘player_dismissed’ column it has either value 0 or name of the player got out at that particular ball. First we will replace all the names with 1 and then apply the cumsum() function on it so we can get the total wickets gone and we will subtract it from 10 to get the ‘wickets_left’ column.



In [115]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype('int')
df['player_dismissed'] = df.groupby('match_id').cumsum()['player_dismissed']
df['wickets_left'] = 10 - df['player_dismissed']

In [116]:
df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,ball_num,ball_left,wicket,wickets_left,current_run_rate,last_five,current_score,ball_no,balls_bowled,balls_left
0,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.1,India,Australia,RD Gaikwad,Shubman Gill,...,1,299,0,9,24.000000,NaN,4,1,1,299
1,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.2,India,Australia,RD Gaikwad,Shubman Gill,...,2,298,0,8,12.000000,NaN,4,2,2,298
2,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.3,India,Australia,RD Gaikwad,Shubman Gill,...,3,297,0,7,16.000000,NaN,8,3,3,297
3,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.4,India,Australia,RD Gaikwad,Shubman Gill,...,4,296,0,6,12.000000,NaN,8,4,4,296
4,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.5,India,Australia,RD Gaikwad,Shubman Gill,...,5,295,0,5,9.600000,NaN,8,5,5,295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265098,64814,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,...,6,42,8,-567,5.023256,26.0,470,6,258,42
1265099,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,...,1,41,8,-568,5.027027,27.0,471,1,259,41
1265100,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,...,2,40,9,-569,5.007692,23.0,471,2,260,40
1265101,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,...,3,39,9,-570,5.034483,25.0,473,3,261,39


In [117]:
df['crr']=(df['current_score']*6)/df['balls_bowled']

In [118]:
df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,ball_left,wicket,wickets_left,current_run_rate,last_five,current_score,ball_no,balls_bowled,balls_left,crr
0,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.1,India,Australia,RD Gaikwad,Shubman Gill,...,299,0,9,24.000000,NaN,4,1,1,299,24.000000
1,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.2,India,Australia,RD Gaikwad,Shubman Gill,...,298,0,8,12.000000,NaN,4,2,2,298,12.000000
2,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.3,India,Australia,RD Gaikwad,Shubman Gill,...,297,0,7,16.000000,NaN,8,3,3,297,16.000000
3,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.4,India,Australia,RD Gaikwad,Shubman Gill,...,296,0,6,12.000000,NaN,8,4,4,296,12.000000
4,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.5,India,Australia,RD Gaikwad,Shubman Gill,...,295,0,5,9.600000,NaN,8,5,5,295,9.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265098,64814,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,...,42,8,-567,5.023256,26.0,470,6,258,42,10.930233
1265099,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,...,41,8,-568,5.027027,27.0,471,1,259,41,10.911197
1265100,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,...,40,9,-569,5.007692,23.0,471,2,260,40,10.869231
1265101,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,...,39,9,-570,5.034483,25.0,473,3,261,39,10.873563


### Now we need a column that has total runs scored in last five overs. Obviously we will have null values in this column for first 5 overs.



In [119]:
# Calculate the total runs (including 'runs_off_bat' and 'extras') scored in the last five overs for each match
df['last_five'] = df.groupby('match_id')[['runs_off_bat', 'extras']].rolling(window=5).sum().reset_index(level=0, drop=True).sum(axis=1)

In [120]:
df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,ball_left,wicket,wickets_left,current_run_rate,last_five,current_score,ball_no,balls_bowled,balls_left,crr
0,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.1,India,Australia,RD Gaikwad,Shubman Gill,...,299,0,9,24.000000,0.0,4,1,1,299,24.000000
1,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.2,India,Australia,RD Gaikwad,Shubman Gill,...,298,0,8,12.000000,0.0,4,2,2,298,12.000000
2,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.3,India,Australia,RD Gaikwad,Shubman Gill,...,297,0,7,16.000000,0.0,8,3,3,297,16.000000
3,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.4,India,Australia,RD Gaikwad,Shubman Gill,...,296,0,6,12.000000,0.0,8,4,4,296,12.000000
4,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.5,India,Australia,RD Gaikwad,Shubman Gill,...,295,0,5,9.600000,8.0,8,5,5,295,9.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265098,64814,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,...,42,8,-567,5.023256,3.0,470,6,258,42,10.930233
1265099,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,...,41,8,-568,5.027027,3.0,471,1,259,41,10.911197
1265100,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,...,40,9,-569,5.007692,1.0,471,2,260,40,10.869231
1265101,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,...,39,9,-570,5.034483,3.0,473,3,261,39,10.873563


### Now we have to create a last column which would be our target column. Total runs scored in that innings.


In [121]:
# Group by 'match_id', sum 'runs' and 'extras', and merge with the original DataFrame
df['total_runs'] = df.groupby('match_id')['runs_off_bat'].transform('sum') + df.groupby('match_id')['extras'].transform('sum')

In [122]:
df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,ball_left,wicket,wickets_left,current_run_rate,last_five,current_score,ball_no,balls_bowled,balls_left,crr
0,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.1,India,Australia,RD Gaikwad,Shubman Gill,...,299,0,9,24.000000,0.0,4,1,1,299,24.000000
1,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.2,India,Australia,RD Gaikwad,Shubman Gill,...,298,0,8,12.000000,0.0,4,2,2,298,12.000000
2,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.3,India,Australia,RD Gaikwad,Shubman Gill,...,297,0,7,16.000000,0.0,8,3,3,297,16.000000
3,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.4,India,Australia,RD Gaikwad,Shubman Gill,...,296,0,6,12.000000,0.0,8,4,4,296,12.000000
4,1389389,2023/24,2023-09-24,"Holkar Cricket Stadium, Indore",1,0.5,India,Australia,RD Gaikwad,Shubman Gill,...,295,0,5,9.600000,8.0,8,5,5,295,9.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265098,64814,2002/03,2002-12-29,"McLean Park, Napier",2,42.6,India,New Zealand,J Srinath,Z Khan,...,42,8,-567,5.023256,3.0,470,6,258,42,10.930233
1265099,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.1,India,New Zealand,Z Khan,J Srinath,...,41,8,-568,5.027027,3.0,471,1,259,41,10.911197
1265100,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.2,India,New Zealand,J Srinath,Z Khan,...,40,9,-569,5.007692,1.0,471,2,260,40,10.869231
1265101,64814,2002/03,2002-12-29,"McLean Park, Napier",2,43.3,India,New Zealand,Z Khan,A Nehra,...,39,9,-570,5.034483,3.0,473,3,261,39,10.873563


In [123]:
final_df=df[['batting_team','bowling_team','venue','current_score','balls_left','wickets_left','crr','last_five','total_runs']]

In [124]:
final_df

,batting_team,bowling_team,venue,current_score,balls_left,wickets_left,crr,last_five,total_runs
0,India,Australia,"Holkar Cricket Stadium, Indore",4,299,9,24.000000,0.0,616
1,India,Australia,"Holkar Cricket Stadium, Indore",4,298,8,12.000000,0.0,616
2,India,Australia,"Holkar Cricket Stadium, Indore",8,297,7,16.000000,0.0,616
3,India,Australia,"Holkar Cricket Stadium, Indore",8,296,6,12.000000,0.0,616
4,India,Australia,"Holkar Cricket Stadium, Indore",8,295,5,9.600000,8.0,616
...,...,...,...,...,...,...,...,...,...
1265098,India,New Zealand,"McLean Park, Napier",470,42,-567,10.930233,3.0,473
1265099,India,New Zealand,"McLean Park, Napier",471,41,-568,10.911197,3.0,473
1265100,India,New Zealand,"McLean Park, Napier",471,40,-569,10.869231,1.0,473
1265101,India,New Zealand,"McLean Park, Napier",473,39,-570,10.873563,3.0,473


### Now we will drop all the columns which we dont want to have for our model and keep those which we created just now. Also we will shuffle the data to avoid any kind of bias.

In [125]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
venue            0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
total_runs       0
dtype: int64

In [126]:
final_df.dropna(inplace=True)

In [127]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
venue            0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
total_runs       0
dtype: int64

In [128]:
final_df=final_df.sample(final_df.shape[0])

In [129]:
final_df

,batting_team,bowling_team,venue,current_score,balls_left,wickets_left,crr,last_five,total_runs
1109087,South Africa,Australia,SuperSport Park,399,108,-478,12.468750,7.0,436
1092403,England,Ireland,"Civil Service Cricket Club, Stormont",225,56,-246,5.532787,9.0,564
477321,South Africa,England,Newlands,444,59,-520,11.053942,3.0,473
204348,Zimbabwe,Pakistan,Rawalpindi Cricket Stadium,135,111,-185,4.285714,5.0,561
590587,England,India,Sophia Gardens,394,150,-459,15.760000,1.0,465
...,...,...,...,...,...,...,...,...,...
464355,Scotland,United Arab Emirates,"Grange Cricket Club Ground, Raeburn Place",38,262,-28,6.000000,4.0,556
221002,South Africa,England,Newlands,424,118,-482,13.978022,3.0,517
1243890,South Africa,Canada,"Buffalo Park, East London",62,203,-94,3.835052,2.0,390
58743,West Indies,South Africa,"Buffalo Park, East London",278,36,-266,6.318182,10.0,622


### With this we end out feature extraction part of the project. So after lot of work we finally have the exact required data we wanted at the start.

### So lets now begin with model building process. For that first we will divide our dataset in training set and testing set using train_test_split module of sklearn library



In [130]:
#Train-Test-Split

X=final_df.drop(columns=['total_runs'])
y=final_df['total_runs']
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [131]:
X_train

,batting_team,bowling_team,venue,current_score,balls_left,wickets_left,crr,last_five
304163,India,Australia,Sydney Cricket Ground,374,161,-441,16.143885,2.0
182557,Bangladesh,Zimbabwe,Harare Sports Club,199,60,-236,4.975000,5.0
959796,Bangladesh,Pakistan,Multan Cricket Stadium,61,206,-88,3.893617,2.0
628329,South Africa,India,SuperSport Park,269,23,-270,5.826715,7.0
1034453,Scotland,South Africa,"Warner Park, Basseterre",115,59,-234,2.863071,2.0
...,...,...,...,...,...,...,...,...
413852,Papua New Guinea,United Arab Emirates,Sheikh Zayed Stadium,8,290,0,4.800000,3.0
1119281,South Africa,Sri Lanka,Western Australia Cricket Association Ground,319,179,-407,15.818182,7.0
305284,Sri Lanka,New Zealand,Bay Oval,385,214,-383,26.860465,2.0
888027,Pakistan,West Indies,New Wanderers Stadium,194,205,-303,12.252632,0.0


### Some preprocessing steps are required here. We will apply one hot encoding on the categorical features (batting_team, bowling_team and city) then we will create a pipleline which would be having our ml model. Also we will apply scaling on our data so that all values come in one range.


### Here four our model I will be using ```xgboost algorithm.```

In [132]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [133]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','venue'])
]
,remainder='passthrough')

In [134]:
pipe=Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=300,learning_rate=0.2,max_depth=12,random_state=1))

])

In [135]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['batting_team',
                                                   'bowling_team',
                                                   'venue'])])),
                ('step2', StandardScaler()),
                ('step3',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.2,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=12, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=300, n_jobs=None,
                              num_parallel_tree=None, random_state=1, ...))])

In [136]:
Y_pred=pipe.predict(X_test)

In [137]:
print(r2_score(Y_test,Y_pred))
print(mean_absolute_error(Y_test,Y_pred))

0.9058203725995297
22.004191226901863


In [140]:
import pickle

In [141]:
pickle.dump(pipe,open('pipe.pkl','wb'))